In [22]:
import torch
from fastai.vision import *
from fastai.callbacks import *
from PIL import Image
import torchvision.transforms.functional as TTF

In [23]:
# loading the learner which we have trained using classifier notebook
learn = load_learner("path of folder containing exported model", "model.pkl")

In [25]:
# shifting model to cpu 
my_model = learn.model.cpu()
softmaxer = torch.nn.Softmax(dim=1)

In [28]:
# predcting using loaded model to check whether it's working perfectly fine or not.
my_model.eval()
image = Image.open("path of image")
image = image.resize((448, 448))
x = TTF.to_tensor(image).div_(255)
x = TTF.normalize(x, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
x.unsqueeze_(0)
print(x.size())
raw_out = my_model(x)
print(softmaxer(raw_out))

torch.Size([1, 3, 448, 448])
tensor([[0.3688, 0.6312]], grad_fn=<SoftmaxBackward>)


In [7]:
# tracing the graph of model then converting to serialized form and then saving it
traced_cell = torch.jit.trace(my_model, x)
traced_cell.save('model.zip')

In [14]:
# it shows how to convert a function to torchscript which can be used in C++
@torch.jit.script
def normalize(inp):
    dtype = inp.dtype
    inp.sub_(torch.tensor([[[0.485]],[[0.456]], [[0.406]]], dtype=dtype).unsqueeze_(0)).div_(torch.tensor([[[0.229]],[[0.224]], [[0.225]]], dtype=dtype).unsqueeze_(0))
    return inp

normalize.save("normalize1.pt")